This notebook presents some code to compute some basic baselines.

In particular, it shows how to:
1. Use the provided validation set
2. Compute the top-30 metric
3. Save the predictions on the test in the right format for submission

In [1]:
%pylab inline --no-import-all

import os
import os
import sys
import inspect
from pathlib import Path

CURR_DIR = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
PARENT_DIR = os.path.dirname(CURR_DIR)
sys.path.insert(0, "/home/mila/s/sara.ebrahim-elkafrawy/scratch/ecosystem_project/geolife_kaggle")

print(PARENT_DIR)
print(CURR_DIR)
import pandas as pd


# Change this path to adapt to where you downloaded the data
DATA_PATH = Path("/network/scratch/s/sara.ebrahim-elkafrawy")

# Create the path to save submission files
SUBMISSION_PATH = Path("/network/scratch/s/sara.ebrahim-elkafrawy/ecosystem_project/")
os.makedirs(SUBMISSION_PATH, exist_ok=True)

Populating the interactive namespace from numpy and matplotlib
/network/scratch/s/sara.ebrahim-elkafrawy/ecosystem_project/geolife_kaggle
/network/scratch/s/sara.ebrahim-elkafrawy/ecosystem_project/geolife_kaggle/notebooks


We also load the official metric, top-30 error rate, for which we provide efficient implementations:

In [2]:
# from ..metrics
from metrics import top_30_error_rate
help(top_30_error_rate)

Help on function top_30_error_rate in module metrics:

top_30_error_rate(y_true, y_score)
    Computes the top-30 error rate.
    
    Parameters
    ----------
    y_true: 1d array, [n_samples]
        True labels.
    y_score: 2d array, [n_samples, n_classes]
        Scores for each label.
    
    Returns
    -------
    float:
        Top-30 error rate value.
    
    Notes
    -----
    Complexity: :math:`O( n_\text{samples} \times n_\text{classes} )`.



In [3]:
from metrics import top_k_error_rate_from_sets
help(top_k_error_rate_from_sets)

Help on function top_k_error_rate_from_sets in module metrics:

top_k_error_rate_from_sets(y_true, s_pred)
    Computes the top-k error rate from predicted sets.
    
    Parameters
    ----------
    y_true: 1d array, [n_samples]
        True labels.
    s_pred: 2d array, [n_samples, k]
        Previously computed top-k sets for each sample.
    
    Returns
    -------
    float:
        Error rate value.



For submissions, we will also need to predict the top-30 sets for which we also provide an efficient implementation:

In [4]:
from metrics import predict_top_30_set
help(predict_top_30_set)

Help on function predict_top_30_set in module metrics:

predict_top_30_set(y_score)
    Predicts the top-30 sets from scores.
    
    Parameters
    ----------
    y_score: 2d array, [n_samples, n_classes]
        Scores for each sample and label.
    
    Returns
    -------
    2d array, [n_samples, 30]:
        Predicted top-30 sets for each sample.
    
    Notes
    -----
    Complexity: :math:`O( n_\text{samples} \times n_\text{classes} )`.



We also provide an utility function to generate submission files in the right format:

In [5]:
from submission import generate_submission_file
help(generate_submission_file)

Help on function generate_submission_file in module submission:

generate_submission_file(filename, observation_ids, s_pred)
    Generate submission file for Kaggle
    
    Parameters
    ----------
    filename : string
        Submission filename.
    observation_ids : 1d array-like
        Test observations ids
    s_pred : list of 1d array-like
        Set predictions for test observations.



# Observation data loading

We first need to load the observation data:

In [6]:
df_obs_fr = pd.read_csv(DATA_PATH / "observations" / "observations_fr_train.csv", sep=";", index_col="observation_id")
df_obs_us = pd.read_csv(DATA_PATH / "observations" / "observations_us_train.csv", sep=";", index_col="observation_id")
df_obs = pd.concat((df_obs_fr, df_obs_us))

Then, we retrieve the train/val split provided:

In [7]:
obs_id_train = df_obs.index[df_obs["subset"] == "train"].values
obs_id_val = df_obs.index[df_obs["subset"] == "val"].values

y_train = df_obs.loc[obs_id_train]["species_id"].values
y_val = df_obs.loc[obs_id_val]["species_id"].values

n_val = len(obs_id_val)
print("Validation set size: {} ({:.1%} of train observations)".format(n_val, n_val / len(df_obs)))

Validation set size: 40080 (2.5% of train observations)


We also load the observation data for the test set:

In [8]:
df_obs_fr_test = pd.read_csv(DATA_PATH / "observations" / "observations_fr_test.csv", sep=";", index_col="observation_id")
df_obs_us_test = pd.read_csv(DATA_PATH / "observations" / "observations_us_test.csv", sep=";", index_col="observation_id")

df_obs_test = pd.concat((df_obs_fr_test, df_obs_us_test))

obs_id_test = df_obs_test.index.values

print("Number of observations for testing: {}".format(len(df_obs_test)))

df_obs_test.head()

Number of observations for testing: 36421


,latitude,longitude
observation_id,,
10782781,43.601788,6.940195
10364138,46.241711,0.683586
10692017,45.181095,1.533459
10222322,46.938450,5.298678
10241950,45.017433,0.960736


# Sample submission file

In this section, we will demonstrate how to generate the sample submission file provided.

To do so, we will use the function `generate_submission_file` from `GLC.submission`.

The sample submission consists in always predicting the first 30 species for all the test observations:

In [9]:
first_30_species = np.arange(30)
s_pred = np.tile(first_30_species[None], (len(df_obs_test), 1))

In [10]:
s_pred.shape

(36421, 30)

In [11]:
s_pred[0]

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29])

We can then generate the associated submission file using:

In [12]:
generate_submission_file(SUBMISSION_PATH / "sample_submission.csv", df_obs_test.index, s_pred)

# Constant baseline: 30 most observed species

The first baseline consists in predicting the 30 most observed species on the train set which corresponds exactly to the "Top-30 most present species":

In [13]:
species_distribution = df_obs.loc[obs_id_train]["species_id"].value_counts(normalize=True)
top_30_most_observed = species_distribution.index.values[:30]

As expected, it does not perform very well on the validation set:

In [14]:
s_pred = np.tile(top_30_most_observed[None], (n_val, 1))
s_pred.shape
score = top_k_error_rate_from_sets(y_val, s_pred)
print("Top-30 error rate: {:.1%}".format(score))

Top-30 error rate: 93.5%


In [15]:
y_val.shape, s_pred.shape

((40080,), (40080, 30))

In [16]:
y_val.shape, np.max(y_val)

((40080,), 17006)

In [17]:
np.max(s_pred)

5612

We will however generate the associated submission file on the test using:

In [18]:
# Compute baseline on the test set
n_test = len(df_obs_test)
s_pred = np.tile(top_30_most_observed[None], (n_test, 1))

# Generate the submission file
generate_submission_file(SUBMISSION_PATH / "constant_top_30_most_present_species_baseline.csv", df_obs_test.index, s_pred)

# Random forest on environmental vectors

A classical approach in ecology is to train Random Forests on environmental vectors.

We show here how to do so using [scikit-learn](https://scikit-learn.org/).

We start by loading the environmental vectors:

In [19]:
df_env = pd.read_csv(DATA_PATH / "pre-extracted" / "environmental_vectors.csv", sep=";", index_col="observation_id")

X_train = df_env.loc[obs_id_train].values
X_val = df_env.loc[obs_id_val].values
X_test = df_env.loc[obs_id_test].values

/home/mila/s/sara.ebrahim-elkafrawy/.conda/envs/glc/lib/python3.6/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


Then, we need to handle properly the missing values.

For instance, using `SimpleImputer`:

In [20]:
from sklearn.impute import SimpleImputer
imp = SimpleImputer(
    missing_values=np.nan,
    strategy="constant",
    fill_value=np.finfo(np.float32).min,
)
imp.fit(X_train)

X_train = imp.transform(X_train)
X_val = imp.transform(X_val)
X_test = imp.transform(X_test)

We can now start training our Random Forest (as there are a lot of observations, over 1.8M, this can take a while):

In [21]:
from sklearn.ensemble import RandomForestClassifier
est = RandomForestClassifier(n_estimators=16, max_depth=10, n_jobs=-1)
# est.fit(X_train, y_train)
est.fit(X_val, y_val)

RandomForestClassifier(max_depth=10, n_estimators=16, n_jobs=-1)

As there are a lot of classes (over 17K), we need to be cautious when predicting the scores of the model.

This can easily take more than 5Go on the validation set.

For this reason, we will be predict the top-30 sets by batches using the following generic function:

In [22]:
def batch_predict(predict_func, X, batch_size=1024):
    res = predict_func(X[:1])
    n_samples, n_outputs, dtype = X.shape[0], res.shape[1], res.dtype
    
    preds = np.empty((n_samples, n_outputs), dtype=dtype)
    
    for i in range(0, len(X), batch_size):
        X_batch = X[i:i+batch_size]
        preds[i:i+batch_size] = predict_func(X_batch)
            
    return preds

We can know compute the top-30 error rate on the validation set:

In [30]:
y_val.shape, s_val.shape

((40080,), (40080, 30))

In [23]:
def predict_func(X):
    y_score = est.predict_proba(X)
    s_pred = predict_top_30_set(y_score)
#     print(s_pred.shape)
    return s_pred
s_val = batch_predict(predict_func, X_val, batch_size=1024)
score_val = top_k_error_rate_from_sets(y_val, s_val)
print("Top-30 error rate: {:.1%}".format(score_val))

False
[   7   15   29   32   34   51   56   97  188  248  343  421  423  447
  475  533  657  702  719  778  852  918  972 1088 1141 1375 1991 2585
 4615 5425]
[   7   15   29   32   34   51   56   97  188  248  343  421  423  447
  475  533  657  702  719  778  852  918  972 1088 1141 1375 1991 2585
 4615 5425]
False
[   5  111  811 1890 1934 2042 2061 2132 2520 2523 2528 2537 2543 2638
 2648 2652 2699 2740 2767 2834 2843 2865 3030 3043 3111 3117 3379 4123
 4512 6003]
[   5  111  811 1890 1934 2042 2061 2132 2520 2523 2528 2537 2543 2638
 2648 2652 2699 2740 2767 2834 2843 2865 3030 3043 3111 3117 3379 4123
 4512 6003]
False
[  35  192  403  690  830 1119 1162 1178 1300 1368 1635 1647 1674 1728
 1827 1851 2569 2742 2967 3617 3738 3830 3833 3938 3941 4298 4311 4553
 5581 5955]
[  35  192  403  690  830 1119 1162 1178 1300 1368 1635 1647 1674 1728
 1827 1851 2569 2742 2967 3617 3738 3830 3833 3938 3941 4298 4311 4553
 5581 5955]
False
[ 662 1806 1968 2055 2132 2212 2527 2565 2570 2612 2

We now predict the top-30 sets on the test data and save them in a submission file:

In [24]:
# Compute baseline on the test set
s_pred = batch_predict(predict_func, X_test, batch_size=1024)

# Generate the submission file
generate_submission_file(SUBMISSION_PATH / "random_forest_on_environmental_vectors.csv", df_obs_test.index, s_pred)

False
[  16   32  180  221  223  351  459  464  480  482  506  537  546  650
  849  939 1013 1397 1425 1584 1682 1731 2132 2426 2638 2700 2767 2844
 2865 2998]
[  16   32  180  221  223  351  459  464  480  482  506  537  546  650
  849  939 1013 1397 1425 1584 1682 1731 2132 2426 2638 2700 2767 2844
 2865 2998]
False
[  5  33  50  72  77  95 163 171 206 231 263 328 386 388 395 422 427 489
 493 511 516 519 521 544 596 644 708 770 788 860]
[  5  33  50  72  77  95 163 171 206 231 263 328 386 388 395 422 427 489
 493 511 516 519 521 544 596 644 708 770 788 860]
False
[   7    8   29   32  113  117  160  178  236  248  278  313  394  421
  425  449  475  486  528  546  581  613  634  638  719  729  826  854
  972 1013]
[   7    8   29   32  113  117  160  178  236  248  278  313  394  421
  425  449  475  486  528  546  581  613  634  638  719  729  826  854
  972 1013]
False
[ 12  50  64  72  77 157 159 167 171 206 277 314 394 427 448 485 493 516
 596 644 663 697 708 763 770 788 871 882 

In [25]:
import torch
k = 4
x = np.array([3, 4, 2, 1, 10, 8])
x_ids = np.argpartition(x, k)[-k:]
x_ids

array([0, 1, 5, 4])

In [26]:
y = torch.Tensor([3, 4, 2, 1, 10, 8])
vals, idx = torch.topk(y, k)
vals.type(torch.int), idx

(tensor([10,  8,  4,  3], dtype=torch.int32), tensor([4, 5, 1, 0]))

In [27]:
idx.flip(dims=[0])

tensor([0, 1, 5, 4])

In [28]:
tmp_np = idx.numpy()
(tmp_np == x_ids).all()

False